In [1]:
import numpy as np
import cv2
import pathlib
import os
from scipy.linalg import null_space
import sympy
import time

class Camera:
    
    def __init__(self, val):
        self.img_num = val # カメラ番号（int;コンストラクタ）
        
        f = 8000/3
        cx = 1920/2
        cy = 1080/2
        A = np.zeros((3,3))
        A[0,0] = f
        A[0,2] = cx
        A[1,1] = f
        A[1,2] = cy
        A[2,2] = 1
        
        self.A = A # 内部パラメータ(ndarray)後から更新

    def img_load(self):
        folder_path = "image"
        file_path = os.path.join(folder_path, str(self.img_num) + ".png")
        img = cv2.imread(file_path, 1)# BGRで読み込み
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.flip(img, 1)
        self.img = img # 画像(ndarray)
        
    def contour_extraction(self):
        
        color_arr = np.array([[255,0,0],[0,255,0],[0,0,255],
                             [255,255,0],[255,0,255],[0,255,255],
                             [127,127,127],[127,0,127],[0,127,127]],dtype = np.int16)
        masks = np.ones((self.img.shape[0], self.img.shape[1], 9), dtype=np.uint8)
        
        for i, color in enumerate(color_arr):
            lower = np.clip(color, 0, 255)
            upper = np.clip(color, 0, 255)
            img_mask = cv2.inRange(self.img, lower, upper)
            masks[:,:,i] = img_mask
        
        #self.masks = masks # 色ごとのマスク(nd.array)
        
        contour_list = []

        # 色ごとに輪郭（閉曲線）を抽出
        for i in range(masks.shape[2]):
            contours, hierarchy = cv2.findContours(masks[:,:,i],cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
            contour_list += contours
        self.contour_list = contour_list # 輪郭のリスト(list,ndarray)

        #self.frag_list = contours2fragments(self.contour_list) # フラグメントのリスト(list,ndarray)

    def para_load(self):

        folder_path = pathlib.Path("view_mat")
        file_path = os.path.join(folder_path, str(self.img_num)+".csv")
        self.Rt = np.loadtxt(file_path, delimiter="\t")
        self.P = np.dot(self.A, self.Rt[0:3,0:4])
        
        folder_path = pathlib.Path("cam_pos_rot")
        file_path = os.path.join(folder_path, str(self.img_num)+".csv")
        self.cam_world_cood = -np.dot(self.Rt[0:3,0:3].T, self.Rt[0:3,3])

In [2]:
# 曲線分割
def split_list(contour_length, max_frag_len=100, min_frag_len=40, min_overrap=10):
    
    # 輪郭のフラグメントの位置を指定(最小40 pixl)
    if contour_length > max_frag_len:
        pass
    
    elif contour_length < min_frag_len:
        return None
    
    elif contour_length == min_frag_len:
        return [[0,min_frag_len-1]]
    
    else:
        max_frag_len = contour_length
    
    step0 = np.random.randint(min_frag_len, max_frag_len) # 一つ目のフラグメントの長さ（40から100）
    sep_list = [[0,step0]]
    back = np.random.randint(min_overrap, step0-1) # フラグメントを重ねるために戻す分を決める（最小10 pixl）
    next_start = step0 - back
    
    while True:
        
        # 戻った分(back)より進む
        if back+1 > min_frag_len:
            step = np.random.randint(back+1, max_frag_len)
        else:
            step = np.random.randint(min_frag_len, max_frag_len)

        full_length = next_start + step
        sept = [next_start, full_length]
        sep_list.append(sept)
        back = np.random.randint(10, step-1)
        next_start = full_length - back

        # 終了判定
        if full_length > contour_length:
            break
    
    # 超過した分戻す（長さはそのまま）
    difference = sep_list[-1][1] - (contour_length-1)
    sep_list[-1][0] -= difference
    sep_list[-1][1] -= difference
    
    return sep_list


def contours_split(contour):
    
    #contour.shape == (N, 2)
    contour_length = contour.shape[0]
    sp_list = split_list(contour_length)
    
    if sp_list == None:
        return None
    
    frag_list = []
    # 位置のリスト通りにスライス
    for sp in sp_list:
        #print(sp)
        frag_list.append(contour[sp[0]:sp[1],:])

    return frag_list


def all_fraged(contours_list):
    
    # 輪郭のリストからフラグメントのリストを得る
    frags_list = []

    #for i in contours_list:
    #temp_list = []
    frags = []
    for j in contours_list:
        temp_frags = contours_split(j.squeeze())
            
        if temp_frags != None:
            frags += temp_frags

    #if frags != []:
    #    frags_list.append(frags)
    
    return frags

#def frag_list_fraged(frags_list):# frags_list[色][輪郭][sep][座標]
#    img_frag_list = []
#    for frag in frags_list:
#        color_frag = all_fraged(frag)
#        img_frag_list.append(color_frag)
#    return img_frag_list

In [21]:
# camera対応
def dim3_distance(vec1, vec2):
    return sum((vec1-vec2)**2)

def camera_correspondence(cam_list):
    vec_list = []
    for i,cam in enumerate(cam_list):
        vec_list.append(cam_list[i].cam_world_cood)
        #print(cam_list[i].cam_world_cood)
    pair_list = []
    for i, vec1 in enumerate(vec_list):
        for j, vec2 in enumerate(vec_list):
            if i == j or i > j :
                continue
            elif dim3_distance(vec1,vec2) < 200:
                pair_list.append((i,j))
    
    return pair_list

def cam_pos_mean(cam_list):
    _cam_pos = np.zeros(3)
    for cam in cam_list:
        _cam_pos += cam.cam_world_cood
    cam_mean = _cam_pos/len(cam_list)
    return cam_mean

def vec_L2(vec):
    return np.sum(vec**2)**(1/2)

def cal_angle(cam_pos1, cam_pos2, cam_mean):
    vec1 = cam_pos1-cam_mean
    vec2 = cam_pos2-cam_mean
    cossin = np.dot(vec1, vec2)/(vec_L2(vec1)*vec_L2(vec2))
    angle = np.arccos(cossin)
    return angle

def cal_angle_all(cam_list):
    pair_list = []
    cam_mean = cam_pos_mean(cam_list)
    for i, cam1 in enumerate(cam_list):
        for j, cam2 in enumerate(cam_list):
            if i == j or i > j:
                continue
            cam1_pos = cam1.cam_world_cood
            cam2_pos = cam2.cam_world_cood
            angle = cal_angle(cam1_pos, cam2_pos, cam_mean)
            if angle < 5/9*np.pi:
                pair_list.append((i,j))
    return pair_list

In [22]:
# epipole取得

def SS_mat(vec3):
    vec3 = np.squeeze(vec3)
    SS_mat = np.zeros((3, 3))
    SS_mat[0,1] = -vec3[2]
    SS_mat[0,2] = vec3[1]
    SS_mat[1,0] = vec3[2]
    SS_mat[1,2] = -vec3[0]
    SS_mat[2,0] = -vec3[1]
    SS_mat[2,1] = vec3[0]
    return SS_mat

def FF_mat(A1, A2, Rt1, Rt2):
    P1 = np.dot(A1, Rt1[0:3,0:4])
    P2 = np.dot(A2, Rt2[0:3,0:4])
    cam_pos1 = -np.dot(Rt1[0:3,0:3].T, Rt1[0:3,3])
    cam_pos1 = np.array([cam_pos1[0], cam_pos1[1], cam_pos1[2], 1])
    epipole2 = np.dot(P2, cam_pos1)
    cam_pos2 = -np.dot(Rt2[0:3,0:3].T, Rt2[0:3,3])
    cam_pos2 = np.array([cam_pos2[0], cam_pos2[1], cam_pos2[2], 1])
    epipole1 = np.dot(P1, cam_pos2)
    return epipole1, epipole2, np.dot(SS_mat(epipole2), np.dot(P2, np.linalg.pinv(P1)))

#def contour_disassembly(contour_list):
#    con_dis = []
#    for i in range(len(contour_list)):
#        if contour_list[i] == []:
#            continue
#        which_dis = np.concatenate(contour_list[i])
#        con_dis.append(which_dis)
#        
#    return np.concatenate(con_dis)

In [23]:
# エピポーラ関係
def epilines_para(frags, F):
    
    lines_list = []
    for frag in frags:
        frag_lines = cv2.computeCorrespondEpilines(frag.reshape(-1,1,2), 1,F) # ndarray(フラグメントの座標数,1,3)
        lines_list.append(frag_lines)
    
    return lines_list

#def para2line(parameter):
#    
#    # 一つのパラメータが渡された時を想定
#    line_coode = np.zeros((1920,2), dtype=np.int64)# imgの幅
#    para = np.squeeze(parameter)# 3次ベクトル
#    for x in range(1920):
#        y = int((-para[0]*x - para[2])/para[1])
#        line_coode[x,0] = x
#        line_coode[x,1] = y
#    
#    return line_coode

#def epiline_cal(frag_paras):
#    # 全ての色に対するエピポーラ線の帯の計算
#    # lines[色][フラグメント][線][座標]
#    lines = []
#    for color in frag_paras:
#        temp_color = []
#        
#        for frag in color:
#            temp_line = []
#
#            for point in frag:
#                line = para2line(point)
#                temp_line.append(line)
#
#            temp_color.append(temp_line)
#        lines.append(temp_color)
#        
#    return lines

#def frags_vs_line(img2_frags, frag_epiline):
#    
#    # frag_epiline shape(1920, 2)
#    surport = np.zeros(len(img2_frags))
#    for i in frag_epiline:
#        for j, frag in enumerate(img2_frags):
#            if i in frag:
#                surport[j] += 1
#    
#    return surport

#def pair_frag_idx(img2_frags, frag_epilines):
#    surport = np.zeros(len(img2_frags))
#    for epi in frag_epilines:
#        surport += frags_vs_line(img2_frags, epi)
#        
#    return surport, np.argmax(surport)


In [24]:
def normalization(vec3):
    return vec3[0]/vec3[2], vec3[1]/vec3[2]

In [25]:
# 再構築
def nom_F(F):
    return (1/sum(sum(F**2))**(1/2))*F

def cover_mat(x1, y1, x2, y2):
    return np.array([[x1**2+x2**2, x2*y2, x2, x1*y1, 0, 0, x1, 0, 0],
                    [x2*y2, x1**2+y2**2, y2, 0, x1*y1, 0, 0, x1, 0],
                    [x2, y2, 1, 0, 0, 0, 0, 0, 0],
                    [x1*y1, 0, 0, y1**2+x2**2, x2*y2, x2, y1, 0, 0],
                    [0, x1*y1, 0, x2*y2, y1**2+y2**2, y2, 0, y1, 0],
                    [0, 0, 0, x2, y2, 1, 0, 0, 0],
                    [x1, 0, 0, y1, 0, 0, 1, 0, 0],
                    [0, x1, 0, 0, y1, 0, 0, 1, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0]])

def min_dist(F, pt1, pt2):# pt1が画像2上の点，pt2が画像1上の点
    S0 = 10**10
    x1_ori = pt1[0]
    y1_ori = pt1[1]
    x2_ori = pt2[0]
    y2_ori = pt2[1]
    
    x1 = pt1[0]
    y1 = pt1[1]
    x2 = pt2[0]
    y2 = pt2[1]
    
    x1_tilda = 0
    y1_tilda = 0
    x2_tilda = 0
    y2_tilda = 0
    thita = nom_F(F).flatten()
    it = 0
    while True:
        V_eps = cover_mat(x1, y1, x2, y2)
        eps_ast = np.array([x1*x2 + x2*x1_tilda + x2*x2_tilda,
                           x1*y2 + y2*x1_tilda + x2*y2_tilda,
                           x1 + x1_tilda,
                           y1*x2 + x2*y1_tilda + y1 * x2_tilda,
                           y1* y2 + y2*y1_tilda + y1*y2_tilda,
                           y1 + y1_tilda,
                           x2 + x2_tilda,
                           y2 + y2_tilda,
                           1])
        
        x1_y1_tilda = np.dot(eps_ast, thita) * np.dot(np.array([[thita[0], thita[1], thita[2]], [thita[3], thita[4], thita[5]]]), np.array([x2, y2, 1])) / np.dot(thita, np.dot(V_eps, thita))
        x2_y2_tilda = np.dot(eps_ast, thita) * np.dot(np.array([[thita[0], thita[3], thita[6]], [thita[1], thita[4], thita[7]]]), np.array([x1, y1, 1])) / np.dot(thita, np.dot(V_eps, thita))
        
        x1_tilda = x1_y1_tilda[0]
        y1_tilda = x1_y1_tilda[1]
        x2_tilda = x2_y2_tilda[0]
        y2_tilda = x2_y2_tilda[1]
        
        x1 = x1_ori - x1_tilda
        y1 = y1_ori - y1_tilda
        x2 = x2_ori - x2_tilda
        y2 = y2_ori - y2_tilda
        
        S = x1_tilda**2 + y1_tilda**2 + x2_tilda**2 + y2_tilda**2
        
        if abs(S0 - S) < 0.00001:
            break

        elif it == 20:
            break

        else:
            S0 = S
            it += 1

    return np.array((x1, y1)), np.array((x2, y2))

def Ps(P, pt):
    a = P[0,0] - pt[0]*P[2,0]
    b = P[0,1] - pt[0]*P[2,1]
    c = P[0,2] - pt[0]*P[2,2]
    d = P[0,3]
    e = pt[0]*P[2,3]
    f = P[1,0] - pt[1]*P[2,0]
    g = P[1,1] - pt[1]*P[2,1]
    h = P[1,2] - pt[1]*P[2,2]
    i = P[1,3]
    j = pt[1]*P[2,3]
    return a, b, c, d, e, f, g, h, i, j

def tri(P1, P2, pt1, pt2):
    #x = sympy.Symbol('x')
    #y = sympy.Symbol('y')
    #z = sympy.Symbol('z')
    a1, b1, c1, d1, e1, f1, g1, h1, i1, j1 = Ps(P1, pt1)
    a2, b2, c2, d2, e2, f2, g2, h2, i2, j2 = Ps(P2, pt2)
    T = np.array([[a1, b1, c1],
                 [f1, g1, h1],
                 [a2, b2, c2],
                 [f2, g2, h2]])
    p = np.array([[d1-e1],
                 [i1-j1],
                 [d2-e2],
                 [i2-j2]])
    T_inv = np.linalg.pinv(T)
    result_pt = np.dot(T_inv, -p)
    return result_pt

    #expr1 = a1*x + b1*y + c1*z + d1 - e1
    #expr2 = f1*x + g1*y + h1*z + i1 - j1
    #expr3 = a2*x + b2*y + c2*z + d2 - e2
    #expr4 = f2*x + g2*y + h2*z + i2 - j2
    #print(expr1.subs(sympy.solve([expr2, expr3, expr4])))
    #result = sympy.solve([expr2, expr3, expr4])
    
    #return np.array([result[x], result[y], result[z]])

pt1 = [1920-741,758] # 0
pt2 = [1920-858,834] # 2
F = cam_pairs_F[(0,2)] #0to2
pt2, pt1 = min_dist(F, pt2, pt1)
print(pt1,pt2)

pt1 = np.array([pt1[0],pt1[1],1]) # 0
pt2 = np.array([pt2[0],pt2[1],1]) # 2

np.dot(np.dot(pt2, F),pt1)

tri(cam_list[0].P, cam_list[2].P, pt1, pt2)

In [26]:
import matplotlib.pyplot as plt

In [27]:
cam_list = [Camera(i) for i in range(24)]
for i in range(len(cam_list)):
    cam_list[i].img_load()
    cam_list[i].contour_extraction()
    cam_list[i].para_load()

In [28]:
# エピポール取得
#cam_pairs = cal_angle_all(cam_list)
cam_pairs = camera_correspondence(cam_list)
epipole_dict = {i:[] for i in range(len(cam_list))}
cam_pairs_F = {}
for i in cam_pairs:
    epipole1, epipole2, F = FF_mat(cam_list[i[0]].A, cam_list[i[1]].A, cam_list[i[0]].Rt, cam_list[i[1]].Rt)
    epipole_dict[i[0]].append(normalization(epipole1))
    epipole_dict[i[1]].append(normalization(epipole2))
    cam_pairs_F[i] = F

In [29]:
cam_pairs

[(0, 2),
 (0, 21),
 (1, 6),
 (1, 12),
 (2, 14),
 (3, 6),
 (3, 8),
 (4, 15),
 (4, 22),
 (5, 8),
 (5, 22),
 (7, 17),
 (7, 20),
 (9, 13),
 (9, 18),
 (10, 14),
 (10, 17),
 (11, 19),
 (11, 23),
 (12, 18),
 (13, 15),
 (16, 19),
 (16, 20),
 (21, 23)]

In [30]:
#エピポールと輪郭上の点を結んだ直線と，x軸のなす角
def gene(angles):
    # 正規化
    B = list(map(lambda y:y-min(angles),angles))
    return list(map(lambda y: (y-min(B))/(max(B)-min(B)), B))

def epipole_angle(img_num, epipole_dict):
    cam = cam_list[img_num]
    angle_list = []
    
    for epi in epipole_dict[img_num]:
        epi_angle_list = []
        epi_x = epi[0]
        epi_y = epi[1]
        for contour in cam.contour_list:
            contour_angle_list= []
            for cood in contour:
                x = cood[0][0]
                y = cood[0][1]
                tilt = (y-epi_y)/(x-epi_x)
                angle = np.arctan(tilt)
                contour_angle_list.append(angle)                
            contour_angle_list = gene(contour_angle_list)
            epi_angle_list.append(contour_angle_list)
        angle_list.append(epi_angle_list)
    return angle_list
    
#def expand(idx_l, list_length):
#    del_list = []
#    for i in idx_l:
#        if np.isnan(i):
#            continue
#        if i-2 < 0:
#            del_list.append(list_length + i-2)
#        else:
#            del_list.append(i-2)
#        
#        if i-1 < 0:
#            del_list.append(list_length + i-1)
#        else:
#           del_list.append(i-1)
#        
#        del_list.append(i)
#        
#        if i+1 > list_length-1:
#            del_list.append(i+1-list_length)
#        else:
#            del_list.append(i+1)
#        if i+2 > list_length-1:
#            del_list.append(i+2-list_length)
#        else:
#            del_list.append(i+2)
#    return sorted(list(set(del_list)))

def differential(angles):
    # エピポーラ線に平行な接線をもつ点(前後方微分の正負を比べたほうが良い)
    del_idx = []
    for i in range(len(angles)):
        if np.isnan(angles[i]):
            continue
        if i == len(angles)-1:
            if np.sign(angles[i]-angles[i-1]) != np.sign(angles[0]-angles[i]): #or abs(angles[0]-angles[i-1])/2 < 0.001:
                del_idx.append(i)
        else:
            if np.sign(angles[i]-angles[i-1]) != np.sign(angles[i+1]-angles[i]):# or abs(angles[i+1]-angles[i-1])/2 < 0.001:
                del_idx.append(i)
    #del_idx = expand(del_idx, len(angles))
    
    return del_idx

def marge_del(all_del_list):
    marged = []
    for j in range(len(all_del_list[0])):
        stem = []
        for i in range(len(all_del_list)):
            stem += all_del_list[i][j]
        marged.append(list(set(stem)))
    return marged

def all_D(angles_list):
    # 画像1枚に対して削除リストを作成
    all_del_list = []
    for epi in angles_list:
        epi_del_list = []
        for contour in epi:
            del_idx = differential(contour)
            epi_del_list.append(del_idx)
        all_del_list.append(epi_del_list)
    marged = marge_del(all_del_list)
    return marged

In [31]:
# エピポーラ線との接線で分割
def separate(contour, del_idx):
    # 一つの輪郭に対し削除リストから削除
    start = 0
    newArray = []
    for d in del_idx:
        if contour[start:d] != []:
            if contour[start:d].size != 0:
                newArray.append(contour[start:d])
        start = d+1

    if contour[start:].size != 0:
        newArray.append(contour[start:])
    return newArray

def all_sep(con_list, del_list):
    n_list = []
    for con, del_con in zip(con_list, del_list) :
        n_con = separate(con, del_con)
        if len(n_con) != 0:
            for i in n_con:
                n_list.append(i)
    return n_list

In [32]:
def frag_max_len(frag_list):
    max_len = 0

    for i in frag_list:
        a = len(i)
        if a > max_len:
            max_len = a
    return max_len

def trim_frag_list(frag_list):
    max_len = frag_max_len(frag_list)
    np_frag_list = np.empty([len(frag_list), max_len, 2])
    np_frag_list[:,:] = np.nan
    for i, frag in enumerate(frag_list):
        np_frag_list[i, 0:len(frag),] = np.array(frag)
    return np_frag_list

def trim_all_frag():
    for i in range(len(cam_list)):
        frag_li = cam_list[i].frag_list
        np_frag_list = trim_frag_list(frag_li)
        cam_list[i].frag_list = np_frag_list

In [33]:
# フラグメントのリストを作る
for i in range(len(cam_list)):
    im_del_list = all_D(epipole_angle(i, epipole_dict))# im_del_list[color][contour][del_idx]
    newCon = all_sep(cam_list[i].contour_list, im_del_list)# newCon[color][fragment][coordination]
    cam_list[i].frag_list = all_fraged(newCon)
trim_all_frag()

C:\Users\murata\anaconda3\envs\ana37\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
C:\Users\murata\anaconda3\envs\ana37\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  import sys


In [113]:
cam_list[0].frag_list.shape

(1520, 99, 2)

In [90]:
# 衝突判定
def para2cood_S(para_list):
    cood_S = []
    for a,b,c in para_list:
        if np.isnan(a):
            cood_S.append(np.array([np.nan, np.nan]))
        else:
            cood_S.append(np.array([0, -c/b]))
    return cood_S

def para2cood_F(para_list):
    cood_F = []
    for a,b,c in para_list:
        if np.isnan(a):
            cood_F.append(np.array([np.nan, np.nan]))
        else:
            cood_F.append(np.array([1920, -(1920*a+c)/b]))# 画像の横の大きさ
    return cood_F

def all_pa2co(para_list):
    epi_cood_S = []
    epi_cood_F = []
    for frag in para_list:
        S_cood = para2cood_S(frag.squeeze())
        F_cood = para2cood_F(frag.squeeze())
        epi_cood_S.append(S_cood)
        epi_cood_F.append(F_cood)
    return np.array(epi_cood_S), np.array(epi_cood_F) # epi_cood[frag]

def get_frag_cood(frag_list):
    cood_S = np.array([frag[0] for frag in frag_list])
    cood_F = []
    for frag in frag_list:
        frag = frag[~np.isnan(frag)]
        cood_F.append((frag[-2],frag[-1]))
    cood_F = np.array(cood_F)
    return cood_S, cood_F # cood_S[frag]

def coll_t1_t2(epi_cood_S, epi_cood_F, cood_S, cood_F):
    epi_cood_S_bro = np.array([np.broadcast_to(aa, (len(cood_S), 2)) for aa in epi_cood_S])
    epi_cood_F_bro = np.array([np.broadcast_to(aa, (len(cood_S), 2)) for aa in epi_cood_F])
    #epi_cood_S_bro = np.tile(epi_cood_S, (1,len(cood_S),1)).transpose((1,0,2)) # 遅い
    #epi_cood_F_bro = np.tile(epi_cood_F, (1,len(cood_S),1)).transpose((1,0,2))
    v = cood_S - epi_cood_S_bro
    v2 = cood_F - cood_S
    v1 = epi_cood_F_bro - epi_cood_S_bro
    t1 = np.cross(v, v2)/np.cross(v1, v2)
    t2 = np.cross(v, v1)/np.cross(v1, v2)
    return t1, t2

def coll_det(t1, t2):
    t1_t = np.array((t1 <= 1) & (t1 > 0),dtype=np.int16)
    t2_t = np.array((t2 <= 1) & (t2 > 0),dtype=np.int16)
    count_c = np.array(t1_t + t2_t == 2, dtype=np.int64)
    surport_idx = np.argmax(np.sum(count_c,axis=0))
    return surport_idx

def make_piar_list(epi_cood_S, epi_cood_F, cood_S, cood_F):
    img_list=[]
    for epi_frag_S, epi_frag_F in zip(epi_cood_S, epi_cood_F):
        t1, t2 = coll_t1_t2(epi_frag_S, epi_frag_F, cood_S, cood_F)
        surport_idx = coll_det(t1, t2)
        img_list.append(surport_idx)
    return img_list

for i in cam_pairs_F:
    F = cam_pairs_F[i]
    frags_para12 = epilines_para(cam_list[i[0]].frag_list, F) # frags_para[色][frag]
    frags_para21 = epilines_para(cam_list[i[1]].frag_list, F.T)
    cood_S, cood_F = get_frag_cood(cam_list[i[1]].frag_list)
    epi_cood_S, epi_cood_F = all_pa2co(frags_para12)
    
    img_list1 = make_piar_list(epi_cood_S, epi_cood_F, cood_S, cood_F)

In [91]:
#start_time = time.time()
pair_list = {}
for pair in cam_pairs_F:
    F = cam_pairs_F[pair]
    frags_para12 = epilines_para(cam_list[pair[0]].frag_list, F) # frags_para[色][frag]
    frags_para21 = epilines_para(cam_list[pair[1]].frag_list, F.T)
    cood_S, cood_F = get_frag_cood(cam_list[pair[1]].frag_list)
    epi_cood_S, epi_cood_F = all_pa2co(frags_para12)
    
    img_list1 = make_piar_list(epi_cood_S, epi_cood_F, cood_S, cood_F)
    
    cood_S, cood_F = get_frag_cood(cam_list[pair[0]].frag_list)
    epi_cood_S, epi_cood_F = all_pa2co(frags_para21)
    img_list2 = make_piar_list(epi_cood_S, epi_cood_F, cood_S, cood_F)
    
    pair_list[((pair[0],pair[1]), "F")] = img_list1
    pair_list[((pair[0],pair[1]), "R")] = img_list2
    #elapsed_time = time.time() - start_time
    #print(elapsed_time)
# 直線のパラメータから座標にするとき桁が大きくなりすぎてエラーを吐くことがある（para2cood）

C:\Users\murata\anaconda3\envs\ana37\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in less_equal
C:\Users\murata\anaconda3\envs\ana37\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in greater
C:\Users\murata\anaconda3\envs\ana37\lib\site-packages\ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in less_equal
C:\Users\murata\anaconda3\envs\ana37\lib\site-packages\ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in greater


43.124045610427856
77.61929392814636
132.86641025543213
159.32306241989136
217.23399591445923
242.1095907688141
250.58919501304626
256.03159976005554
257.3545217514038
267.4324779510498
270.32703137397766
312.42782330513
346.5999081134796
349.01810789108276
352.2646133899689
427.8552191257477
491.2849352359772
574.3415341377258
652.3206889629364
663.4704356193542
669.6039769649506
743.0264399051666
781.824075460434
837.0731883049011


In [130]:
pair_list[((0,2),"F")]

[475,
 2,
 331,
 331,
 34,
 94,
 6,
 8,
 24,
 24,
 110,
 110,
 24,
 10,
 331,
 331,
 348,
 348,
 4,
 20,
 21,
 52,
 110,
 26,
 98,
 10,
 24,
 23,
 23,
 23,
 331,
 18,
 4,
 124,
 110,
 110,
 110,
 10,
 10,
 23,
 23,
 23,
 331,
 33,
 4,
 37,
 291,
 110,
 110,
 24,
 10,
 10,
 24,
 22,
 331,
 348,
 68,
 96,
 180,
 24,
 110,
 110,
 110,
 98,
 10,
 24,
 23,
 331,
 348,
 34,
 4,
 5,
 52,
 208,
 110,
 110,
 110,
 24,
 10,
 10,
 10,
 23,
 23,
 23,
 23,
 331,
 348,
 247,
 357,
 68,
 37,
 23,
 10,
 110,
 26,
 26,
 10,
 10,
 23,
 23,
 331,
 348,
 247,
 4,
 96,
 8,
 24,
 110,
 110,
 110,
 110,
 10,
 23,
 51,
 348,
 34,
 95,
 51,
 110,
 110,
 24,
 10,
 23,
 37,
 331,
 331,
 247,
 4,
 4,
 4,
 51,
 291,
 110,
 110,
 110,
 10,
 10,
 10,
 23,
 23,
 51,
 33,
 34,
 4,
 4,
 21,
 180,
 110,
 110,
 98,
 10,
 10,
 24,
 23,
 23,
 21,
 348,
 348,
 95,
 238,
 10,
 110,
 26,
 110,
 24,
 10,
 10,
 10,
 23,
 37,
 37,
 21,
 37,
 247,
 4,
 68,
 23,
 110,
 110,
 110,
 98,
 10,
 10,
 10,
 23,
 8,
 7,
 21,
 34,
 4,
 4,


In [92]:
def FR_cheacker(pair, pair_list, cam_list):
    F = cam_pairs_F[pair[0]]
    if pair[1] == "F":
        frags_para = epilines_para(cam_list[pair[0][0]].frag_list, F) # frags_para[色][frag]
        epi_cood_S, epi_cood_F = all_pa2co(frags_para)
        camL_idx = pair[0][1]
    elif pair[1] == "R":
        frags_para = epilines_para(cam_list[pair[0][1]].frag_list, F.T)
        epi_cood_S, epi_cood_F = all_pa2co(frags_para)
        camL_idx = pair[0][0]
    return epi_cood_S, epi_cood_F, camL_idx

In [115]:
# 線と点の衝突判定
def PL_coll(pair, pair_list, cam_list):
    epi_cood_S, epi_cood_F, camL_idx = FR_cheacker(pair, pair_list, cam_list)
    frag_part = cam_list[camL_idx].frag_list
    im_list = []
    for pair_frag, epi_S_frag, epi_F_frag in zip(pair_list[pair], epi_cood_S, epi_cood_F):
        pts = frag_part[pair_frag] # 対応するフラグメント
        v1 = epi_F_frag - epi_S_frag
        v1_n = (v1[:,0]**2+v1[:,1]**2)**(1/2)
        v1_n = np.stack([v1_n, v1_n], axis=1)
        v1 = v1/v1_n
        v1_bro = np.array([np.broadcast_to(aa, (len(pts), 2)) for aa in v1])
        epi_cood_S_bro = np.array([np.broadcast_to(aa, (len(pts), 2)) for aa in epi_S_frag])
        v2 = pts - epi_cood_S_bro
        v2_n = (v2[:,:,0]**2+v2[:,:,1]**2)**(1/2)
        v2_n = np.stack([v2_n, v2_n], axis=2)
        v2 = v2/v2_n
        con_det = np.cross(v1_bro, v2)
        im_list.append(np.where(np.abs(con_det) <= 0.001))
    return im_list

In [116]:
# 点と線の衝突判定
coll_dict = {}
for pair in pair_list:
    im_list = PL_coll(pair, pair_list, cam_list)
    coll_dict[pair] = im_list

C:\Users\murata\anaconda3\envs\ana37\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in less_equal


In [122]:
len(coll_dict[((0,2),"F")])

1520

In [101]:
def pt_pair(coll_list):
    pool_i = []
    pool_j = []
    pre_i = None
    pre_j = None
    pt = 1
    for i, j in zip(coll_list[0], coll_list[1]):
        if i in pool_i:
            if pt == 1:
                continue
            elif pt == 0:
                if j not in pool_j:
                    pool_i.pop()
                    pool_j.pop()
                    pool_i.append(i)
                    pool_j.append(j)
                else:
                    continue

        elif i not in pool_i:
            if j in pool_j:
                pt = 0
            else:
                pt = 1
            pool_i.append(i)
            pool_j.append(j)
    return np.array([pool_i, pool_j])

In [102]:
# 点の一対一対応
pair_pt = {}
for i in coll_dict:
    im_list = []
    for frag in coll_dict[i]:
        newPair = pt_pair(frag)
        im_list.append(newPair)
    pair_pt[i] = im_list

In [105]:
pair_pt[i][0]

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
        32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
        48, 49, 50, 51, 52, 53, 54, 55, 56, 57],
       [13, 14, 15, 16, 17, 18, 19, 20, 21, 21, 22, 23, 23, 23, 24, 25,
        26, 27, 27, 28, 28, 29, 29, 30, 31, 32, 33, 34, 35, 36, 44, 37,
        38, 40, 47, 48, 50, 51, 52, 53, 54, 56, 57, 56, 58, 58, 59, 60,
        60, 61, 62, 62, 63, 64, 65, 66, 67, 68]], dtype=int64)

In [108]:
def FR_frags(dict_tag):
    if dict_tag[1] == "F":
        part = cam_list[dict_tag[0][0]].frag_list
        counterpart = cam_list[dict_tag[0][1]].frag_list
        return part, counterpart
        
    elif i[1] == "R":
        part = cam_list[dict_tag[0][1]].frag_list
        counterpart = cam_list[dict_tag[0][0]].frag_list
        return part, counterpart

In [127]:
# 座標でdictを作る
coordinate_dict = {}
nan_idx_dict = {}
for i in pair_list:
    pair_coordinate = []
    nan_idx_list = []
    part, counterpart = FR_frags(i)
    index = 0
    for part_frag, pair, pt_idx in zip(part, pair_list[i], pair_pt[i]):
        if pt_idx[0].size != 0:
            pair_coordinate.append((np.array([part_frag[pt_idx[0]], counterpart[pair][pt_idx[1]]])))
        else:
            nan_idx_list.append(index)
        index += 1
    coordinate_dict[i] = pair_coordinate
    nan_idx_dict[i] = nan_idx_list

In [128]:
for i in nan_idx_dict:
    print(len(nan_idx_dict[i]))

0
110
0
0
0
0
0
58
0
0
51
26
2
65
7
0
0
0
87
17
25
3
4
2
0
0
70
243
0
229
0
0
0
0
0
0
0
0
2
17
6
4
0
0
0
0
0
0


In [24]:
def FR_check(dict_tag):
    if dict_tag[1] == "F":
        P1 = cam_list[dict_tag[0][0]].P
        P2 = cam_list[dict_tag[0][1]].P
        F = cam_pairs_F[dict_tag[0]]
        return P1, P2, F
    elif dict_tag[1] == "R":
        P1 = cam_list[dict_tag[0][1]].P
        P2 = cam_list[dict_tag[0][0]].P
        F = cam_pairs_F[dict_tag[0]].T
        return P1, P2, F

def connect_points(pts_list):
    sep_list = []
    root = np.transpose(pts_list[0], (1, 0, 2))
    length = len(root)-1
    sep_list.append(length)
    for i, pt in enumerate(pts_list):
        pt = np.transpose(pt, (1, 0, 2))
        if i == 0:
            continue
        root = np.concatenate([root, pt], 0)
        length += len(pt)
        sep_list.append(length)
    return root, sep_list

def sep_array(tri_pts, sep_list):
    # 一つの輪郭に対し削除リストから削除
    start = 0
    newArray = []
    for d in sep_list:
        if tri_pts[start:d].size != 0:
            newArray.append(tri_pts[start:d])
        start = d

    if tri_pts[start:].size != 0:
        newArray.append(tri_pts[start:])
    return newArray

TDlines = []
for i, j in enumerate(coordinate_dict):
    
    pts = coordinate_dict[j]
    P1_ori, P2_ori, F_ori = FR_check(j)
    #pt, sep_list = connect_points(pts)
    temp_TDlines = []
    for pt in pts:
        pt = np.transpose(pt, (1, 0, 2))
        F = np.broadcast_to(F_ori, (pt.shape[0],3,3))
        P1 = np.broadcast_to(P1_ori, (pt.shape[0],3,4))
        P2 = np.broadcast_to(P2_ori, (pt.shape[0],3,4))
        newcoords= np.array(list(map(min_dist, F, pt[:,1,:], pt[:,0,:])))
        tri_pt = np.array(list(map(tri, P1, P2, newcoords[:,1,:], newcoords[:,0,:])))
        #pts_array = sep_array(tri_pt, sep_list)
        temp_TDlines.append(tri_pt)
    TDlines.append(temp_TDlines)
    print((i+1)/len(coordinate_dict)*100)
    break

In [25]:
TDlines = {}
for i, j in enumerate(coordinate_dict):
    
    pts = coordinate_dict[j]
    P1_ori, P2_ori, F_ori = FR_check(j)
    #pt, sep_list = connect_points(pts)
    temp_TDlines = []
    for pt in pts:
        pt = np.transpose(pt, (1, 0, 2))
        F = np.broadcast_to(F_ori, (pt.shape[0],3,3))
        P1 = np.broadcast_to(P1_ori, (pt.shape[0],3,4))
        P2 = np.broadcast_to(P2_ori, (pt.shape[0],3,4))
        newcoords= np.array(list(map(min_dist, F, pt[:,1,:], pt[:,0,:])))
        tri_pt = np.array(list(map(tri, P1, P2, newcoords[:,1,:], newcoords[:,0,:])))
        #pts_array = sep_array(tri_pt, sep_list)
        temp_TDlines.append(tri_pt)
    TDlines[j] = temp_TDlines
    print((i+1)/len(coordinate_dict)*100)


2.083333333333333
4.166666666666666
6.25
8.333333333333332
10.416666666666668
12.5
14.583333333333334
16.666666666666664
18.75
20.833333333333336
22.916666666666664
25.0
27.083333333333332
29.166666666666668
31.25
33.33333333333333
35.41666666666667
37.5
39.58333333333333
41.66666666666667
43.75
45.83333333333333
47.91666666666667
50.0
52.083333333333336
54.166666666666664
56.25
58.333333333333336
60.416666666666664
62.5
64.58333333333334
66.66666666666666
68.75
70.83333333333334
72.91666666666666
75.0
77.08333333333334
79.16666666666666
81.25
83.33333333333334
85.41666666666666
87.5
89.58333333333334
91.66666666666666
93.75
95.83333333333334
97.91666666666666
100.0


In [26]:
def excluded_Parray(ex_tag):
    P_dict = {}
    for i, cam in enumerate(cam_list):
        if i in ex_tag:
            continue
        P_dict[i] = cam.P
    return P_dict

def dot_P_frag(P, frag):
    repro_frag = []
    for pt in frag:
        repro_pt = np.dot(P, pt)
        repro_pt = np.array(normalization(repro_pt))
        repro_frag.append(repro_pt)
    return np.array(repro_frag)

In [27]:
reprojection_dict = {}
for tag in TDlines:
    temp_reprojection_dict = {}
    P_dict = excluded_Parray(tag[0])
    for P_tag in P_dict:
        P = P_dict[P_tag]
        P_list = []
        for i, frag in enumerate(TDlines[tag]):
            frag = frag.reshape((-1,3))
            frag = np.concatenate([frag, np.ones(len(frag)).reshape((len(frag), 1))],1) # 末尾に1を追加 (X, Y, Z, 1)
            reprojection = dot_P_frag(P, frag)
            P_list.append(reprojection)
        temp_reprojection_dict[P_tag] = P_list
    reprojection_dict[tag] = temp_reprojection_dict

In [28]:
def connect_contour(contour_list):
    con_list = []
    A = np.concatenate(contour_list).reshape((-1, 2))
    return A

In [29]:
#目印　ここから下

In [30]:
def cal_distance(repro_P, contour_P):
    contour_P = connect_contour(contour_P)
    distance_list = []
    for repro_frag in repro_P:
        repro_frag_bro = np.array([np.broadcast_to(aa, (len(contour_P), 2)) for aa in repro_frag])
        distance = (np.sum((contour_P - repro_frag_bro)**2,axis=2))**(1/2)
        distance_list.append(distance)
    return distance_list

def distance_check(distance_list):
    dist_check_list = []
    ac_list = []
    for frag in distance_list:
        ac = np.array((np.min(frag,axis=1)) < 10,dtype=np.int64) # 条件:10 pixel以内
        dist_check_list.append(np.array(sum(ac)/len(ac)))
        ac_list.append(ac)
    return ac_list, dist_check_list

def P_dict_check(repro_dict_taged):
    P_list = []
    P_ac_list = []
    for P_tag in repro_dict_taged:
        repro_P = repro_dict_taged[P_tag]
        contour_P = cam_list[P_tag].contour_list
        distance_list = cal_distance(repro_P, contour_P)
        ac_list, dist_check_list = distance_check(distance_list)
        P_list.append(dist_check_list)
        P_ac_list.append(ac_list)
    P_check = np.array(P_list)
    return P_ac_list, P_check

def P_check_integration(P_check):
    temp_list=[]
    for img in P_check[:]:
        temp = np.array(img > 0.5,dtype=np.int64)# 曲線中の何割が閾値以内か
        temp_list.append(temp)
    check_list = np.sum(np.array(temp_list),axis=0)
    return check_list
#########################################
def ac_list_integration(P_ac_list):
    inter_ac = []
    for j in range(len(P_ac_list[0])):
        temp_array = np.zeros(len(P_ac_list[0][j]))
        for img in P_ac_list:
            temp_array += img[j]
            inter_ac.append(temp_array)
    return inter_ac
    
def gen_support_dict(reprojection_dicte):
    support_dict = {}
    for tag in reprojection_dict:
        repro_dict_taged = reprojection_dict[tag]
        P_ac_list, P_check = P_dict_check(repro_dict_taged)
        check_list = P_check_integration(P_check)
        inter_ac = ac_list_integration(P_ac_list)
        support_dict[tag] = (check_list, inter_ac)
    return support_dict

In [31]:
support_dict = gen_support_dict(reprojection_dict)

KeyboardInterrupt: 

In [ ]:
sup_th = 11 # サポート数
curve_fragment = []
for tag in TDlines:
    if tag[1] == "R":
        continue
    lines_list = TDlines[tag]
    support_list, support_ac = support_dict[tag][0], support_dict[tag][1]
    for frag, sup, sup_ac in zip(lines_list, support_list, support_ac):
        if sup > sup_th:
            frag = np.reshape(frag,(-1, 3))
            frag = np.array([i for i,j in zip(frag, sup_ac >sup_th) if j])
            curve_fragment.append(frag)

from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

fig = plt.figure(figsize = (12, 12))
fig.patch.set_alpha(0.)
# 3DAxesを追加
ax = fig.add_subplot(111, projection='3d')

# 軸ラベルを設定
ax.set_xlabel("x", size = 14)
ax.set_ylabel("z", size = 14)
ax.set_zlabel("y", size = 14)
ax.set_box_aspect((1,1,1))

def plot_graph():
    for frag in curve_fragment:
        if frag.size == 0:
            continue
        #print(frag)
        x = frag[:,0]
        y = frag[:,1]
        z = frag[:,2]
        data = [x,y,z]
        try:
            tck, u= interpolate.splprep(data, k=3)
        except ValueError:
            pass
        except TypeError:
            pass
        new = interpolate.splev(u, tck, der=0)

        ax.plot(new[0], new[2], new[1],"-")

def plt_graph3d(angle):
    ax.view_init(azim=angle*5)
    
# アニメーションを作成
ani = FuncAnimation(
    fig,
    func=plt_graph3d,
    frames=72,
    init_func=plot_graph,
    interval=200
)

# imagemagickで作成したアニメーションをGIFで書き出す
ani.save("rolling.gif", writer="pillow", savefig_kwargs={'transparent': True, 'facecolor': 'none'})
ani.save('anim.mp4', writer="ffmpeg", savefig_kwargs={'transparent': True, 'facecolor': 'none'})

In [ ]:
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

def plot_graph():
    for frag in curve_fragment:
        if frag.size == 0:
            continue
        #print(frag)
        x = frag[:,0]
        y = frag[:,1]
        z = frag[:,2]
        data = [x,y,z]
        try:
            tck, u= interpolate.splprep(data, k=3)
        except ValueError:
            pass
        except TypeError:
            pass
        new = interpolate.splev(u, tck, der=0)

        ax.plot(new[0], new[2], new[1],"-")
        #ax.plot(x, z, y,"-")
%matplotlib notebook
fig = plt.figure(figsize = (12, 12))
fig.patch.set_alpha(0.)
# 3DAxesを追加
ax = fig.add_subplot(111, projection='3d')

# 軸ラベルを設定
ax.set_xlabel("x", size = 14)
ax.set_ylabel("z", size = 14)
ax.set_zlabel("y", size = 14)
ax.set_box_aspect((1,1,1))
#ax.set_xticks(np.arange(-4,4,1))
#ax.set_yticks(np.arange(-4,4,1))
#ax.set_zticks(np.arange(-2,9,1))

plot_graph()